# Atividade para criar uma "Capa da Invisibilidade"


### Explicação


Para esse projeto ser funcional, como é utilizado em muitas transmissões de vídeo em tempo real, será utilizado uma câmera que fará capturas em tempo real.

O que será necessário?

- Uma boa comfiguração para fazer os tratamentos do algoritmo em tempo real
- Uma camera

### Bibliotecas Utilizadas

In [8]:
import time
import cv2 as cv
import numpy as np

### Desenvolvimento

In [9]:
# Armazena o primeiro frame e utiliza como background da imagem resultante


cap = cv.VideoCapture(0)
_, background = cap.read()
time.sleep(2)
_, background = cap.read()

In [10]:
#define all the kernels size


open_kernel = np.ones((5,5),np.uint8)
close_kernel = np.ones((7,7),np.uint8)
dilation_kernel = np.ones((10, 10), np.uint8)


# Função para remover o ruído da mascara
def filter_mask(mask):
    close_mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, close_kernel)
    open_mask = cv.morphologyEx(close_mask, cv.MORPH_OPEN, open_kernel)
    dilation = cv.dilate(open_mask, dilation_kernel, iterations= 1)
    return dilation

In [11]:
while cap.isOpened():
    ret, frame = cap.read()
    
    # converte do espectro RGB para o HSV
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # define os limites inferior e superior para a cor azul
    lower_bound = np.array([50, 80, 50])
    upper_bound = np.array([90, 255, 255])
    
    
    # define os limites inferior e superior para a cor azul
    lower_bound_blue = np.array([50, 80, 80])
    upper_bound_blue = np.array([255, 255, 255])

    # define os limites inferior e superior para a cor vermelha
    lower_bound_red = np.array([50, 50, 80])
    upper_bound_red = np.array([255, 255, 255])


    # procura as cores dentro dos limites definidos
    mask = cv.inRange(hsv, lower_bound_blue, upper_bound_blue)

    # Filtro da mascara
    mask = filter_mask(mask)
 
    # Substitui os frames da capa de invisibilidade pela imagem salva no background
    cloak = cv.bitwise_and(background, background, mask=mask)

    # Cria a mascara "invertida"
    inverse_mask = cv.bitwise_not(mask)
    
    # Aplica a mascara invertida na regiao onde a capa não está presente.
    current_background = cv.bitwise_and(frame, frame, mask=inverse_mask)

    # Combina a regiao da capa com o background para gerar o resultado final
    combined = cv.add(cloak, current_background)

    cv.imshow("Final output", combined)


    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

### Conclusão


Ainda há bastante coisa a ser feita no algoritmo como a criar ferramentas para fazer uma capa da invisibilidade iterativamente, ou seja, uma imagem é capturada e a seguir pode-se mudar os três atributos H, S ou V por meio de input deslizante.

Outra melhoria seria o tratamento dos pixels que estão mortos na imagem final isso se dá por causa da mascara definida que mesmo funcionando não está funcionando 100%.